<a href="https://colab.research.google.com/github/SoybeanLee/Social-Media-mining/blob/main/Soyeon_final_project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages
!pip install pandas openpyxl datasets transformers

import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
from datasets import Dataset, DatasetDict
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer

def preprocess_data(excel_file):
    # Read the Excel file using pandas
    df = pd.read_excel(excel_file)

    # Convert relevant columns to lists, ensuring 'Message' is string type
    # and label columns are numeric, handling potential errors
    df['Message'] = df['Message'].astype(str)  # Force 'Message' to string

    # Ensure label columns are numeric, replacing non-numeric with 0
    for col in ['Strong_emo', 'Strong_pos', 'Strong_neg', 'TMI', 'Polite', 'Impolite']:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    texts = df['Message'].tolist()
    labels = df[['Strong_emo', 'Strong_pos', 'Strong_neg', 'TMI', 'Polite', 'Impolite']].astype(int).values.tolist()

    return Dataset.from_dict({'text': texts, 'label': labels})

# Now you can call the function with your Excel file:
dataset = preprocess_data('/content/380data_onlytrue.xlsx')

# Tokenize the text
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Load the model for multi-label classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6, problem_type="multi_label_classification")

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,  # Further reduced epochs
    per_device_train_batch_size=16,  # Try increasing if memory allows
    # evaluation_strategy="epoch",  # Remove or comment out this line
    save_steps=10_000,
    save_total_limit=2,
    fp16=True,  # Use mixed precision
    # Add the following line if you want to evaluate at the end of each epoch
    # evaluation_strategy = "epoch"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets
)

# Train the model
trainer.train()

# Save the trained model and tokenizer
model.save_pretrained('/content/saved_model')
tokenizer.save_pretrained('/content/saved_model')


# Load previously saved model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained('/content/saved_model')
tokenizer = DistilBertTokenizer.from_pretrained('/content/saved_model')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sl151 (sl151-indiana-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


In [ ]:
# Function to load test data
def load_test_data(excel_file):
    df = pd.read_excel(excel_file)
    return df['Message'].astype(str).tolist() if 'Message' in df.columns else df.iloc[:, 0].astype(str).tolist()

# Load and tokenize test data
test_texts = load_test_data('/content/Voice_only_onlymessage.xlsx')
inputs = tokenizer(test_texts, return_tensors='pt', padding=True, truncation=True, max_length=128)

# Custom thresholds for each label
custom_thresholds = {
    'Strong_emo': 0.5,
    'Strong_pos': 0.4,
    'Strong_neg': 0.25,
    'TMI': 0.4,
    'Polite': 0.25,
    'Impolite': 0.25
}


In [ ]:
# Run inference
import time
import torch
predictions_list = []
total_batches = (len(inputs['input_ids']) + 31) // 32  # Calculate total batches
start_time = time.time()  # Record the start time

with torch.no_grad():
    for batch_idx in range(total_batches):
        start_idx = batch_idx * 32
        end_idx = min(start_idx + 32, len(inputs['input_ids']))
        batch_inputs = {k: v[start_idx:end_idx].to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}

        # Run inference
        outputs = model(**batch_inputs)
        batch_predictions = torch.sigmoid(outputs.logits).cpu()  # Get raw probabilities

        # Apply custom thresholds for each label
        threshold_tensor = torch.tensor(list(custom_thresholds.values())).to(batch_predictions.device)
        batch_predictions = (batch_predictions > threshold_tensor).int()  # Apply custom thresholds

        predictions_list.append(batch_predictions)

        # Calculate time taken for this batch
        elapsed_time = time.time() - start_time
        avg_time_per_batch = elapsed_time / (batch_idx + 1)  # Average time per batch
        batches_remaining = total_batches - (batch_idx + 1)
        estimated_time_left = avg_time_per_batch * batches_remaining  # Estimate time left

        # Print the estimated time left (in seconds)
        print(f'Batch {batch_idx + 1}/{total_batches} processed. Estimated time left: {estimated_time_left:.2f} seconds')

      # Convert predictions to numpy array
predictions = torch.cat(predictions_list).cpu().numpy()

Batch 1/300 processed. Estimated time left: 3946.38 seconds
Batch 2/300 processed. Estimated time left: 4121.05 seconds
Batch 3/300 processed. Estimated time left: 3430.28 seconds
Batch 4/300 processed. Estimated time left: 3055.90 seconds
Batch 5/300 processed. Estimated time left: 2849.96 seconds
Batch 6/300 processed. Estimated time left: 2713.79 seconds
Batch 7/300 processed. Estimated time left: 2681.54 seconds
Batch 8/300 processed. Estimated time left: 2560.57 seconds
Batch 9/300 processed. Estimated time left: 2513.08 seconds
Batch 10/300 processed. Estimated time left: 2431.56 seconds
Batch 11/300 processed. Estimated time left: 2399.49 seconds
Batch 12/300 processed. Estimated time left: 2343.99 seconds
Batch 13/300 processed. Estimated time left: 2320.68 seconds
Batch 14/300 processed. Estimated time left: 2318.23 seconds
Batch 15/300 processed. Estimated time left: 2298.83 seconds
Batch 16/300 processed. Estimated time left: 2255.33 seconds
Batch 17/300 processed. Estimated

In [ ]:
# Convert predictions to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['Strong_emo', 'Strong_pos', 'Strong_neg', 'TMI', 'Polite', 'Impolite'])

# Create a DataFrame with messages and predictions
results_df = pd.DataFrame({
    'Message': test_texts,
    'Strong_emo': predictions_df['Strong_emo'],
    'Strong_pos': predictions_df['Strong_pos'],
    'Strong_neg': predictions_df['Strong_neg'],
    'TMI': predictions_df['TMI'],
    'Polite': predictions_df['Polite'],
    'Impolite': predictions_df['Impolite']
})

# Specify the output file path for the Excel file
output_file_path = '/content/Voiceonly_results1.xlsx'

# Save the DataFrame to an Excel file
results_df.to_excel(output_file_path, index=False)

print(f'Results have been saved to {output_file_path}')

Results have been saved to /content/Voiceonly_results1.xlsx
